# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from fasthtml.common import *
from monsterui.all import *
from memexplatform_obsidian.commons import config, MountPaths, ResponseTypes
from memexplatform.ui.structure import ifhtmx
from memexplatform_obsidian.mdmanager import ObsidianHTMLRenderer, get_subdirs, resolve_note_path
from mistletoe import Document
import urllib
from memexplatform_obsidian.jupyter import render_nb
from typing import Optional
import pathlib
from pathlib import Path
from memexplatform_obsidian.mdmanager import ObsidianPage, guess_mime
from memexplatform_obsidian.datastore import DBStore, FileStore
from io import BytesIO

In [ ]:
# StreamingResponse??

In [ ]:
#| export
def create_app():
    hdrs = Theme.neutral.headers(highlightjs=True)
    Mermaid = MermaidJS()
    Katex = KatexMarkdownJS()
    D3 = Script(src="https://cdn.jsdelivr.net/npm/d3@7")
    PyscriptCSS=Link(rel="stylesheet", href="https://pyscript.net/releases/2025.3.1/core.css", type="text/css"),
    PyscriptJS=Script(src="https://pyscript.net/releases/2025.3.1/core.js", type='module')
    SurrealGlobalAdd = Script("surreal.globalsAdd());");
    Plotly = Script(src="https://cdn.plot.ly/plotly-3.0.1.min.js")
    add_hdrs = [Mermaid, Katex, D3, PyscriptCSS, PyscriptJS, SurrealGlobalAdd, Plotly]
    newhdrs = hdrs +add_hdrs
    mount_routes = []

    app = FastHTML(hdrs=newhdrs, live=True, exts='ws', routes=mount_routes)
    # app.add_middleware(SessionSyncMiddleware)
    rt = app.route
    return app, rt

app, rt = create_app()

In [ ]:
MountPaths.gradio_url.value

'/obsidian_gradio'

In [ ]:
#| export
@rt
def index(request:Request):

    # listteam_comp = Container(Div('Team'), hx_get=f'/portal/list_team', hx_target='#listteam', id='listteam', hx_swap='innerHTML',hx_vals=json.dumps({"folder": str(config.NBPATH)}),  hx_trigger='load')
    comp = Div("Obsidian Companion")
    
    return ifhtmx(request, comp)

In [ ]:
# vault_path = config.OBSIDIAN_VAULT
# file = "Clippings/Fuck%20You%2C%20Show%20Me%20The%20Prompt.%20%E2%80%93%202.md"
# # file = "Fuck%20You%2C%20Show%20Me%20The%20Prompt.%20%E2%80%93%202.md"
# search_term = urllib.parse.unquote(file); print(search_term)
# file_path = vault_path / search_term; file_path
# base = get_subdirs(vault_path)[0]; base
# # next(base.rglob(search_term), None)
# # list(base.rglob("*"))
# # list(base.rglob("*"))
# print(list(base.rglob("*")))
# next(base.rglob(search_term), None)
# file = "Clippings"

In [ ]:
#| export
@rt
def edit(request:Request, session):
    obsidian_url = session['obsidian_url']
    resp = Response(status_code=200)
    # print(url)
    resp.headers["HX-Redirect"] = obsidian_url
    return resp
    

In [ ]:
#| export
@rt
def open(request: Request, session, file: str = "", title: Optional[str]=None):
    VIDEO_EXTS = (".mp4", ".webm", ".ogg", ".mov", ".mkv")
    AUDIO_EXTS = (".mp3", ".wav", ".ogg", ".m4a", ".flac")
    IMAGE_EXTS = (".png", ".jpg", ".jpeg", ".gif", ".svg", ".webp")
    TEXTLIKE_EXTS = (".md", ".qmd", ".canvas", ".base")
    
    # Simple check: if it's HTMX, return HTML; otherwise return raw file
    is_htmx = request.headers.get('HX-Request')
    
    store = DBStore(config) if config.ENABLE_DB_MODE else FileStore(config)
    out = store.query_file(file)
    if out['content'] is None: return Div(f"Path not found: {urllib.parse.unquote(file)}")

    # If it's a directory → list contents
    if out['is_folder']:
        items = []
        for p in out['content']:
            name = p['fname']
            href = p['url']
            items.append(Li(A(name, href=href)))
        return ifhtmx(request,
            Div(
                f"Listing for {out['title']}:",
                Ul(*items)
            )
        )
    elif out['response_type'] == ResponseTypes.html:
        html = out['content']
        session['obsidian_url'] = out['obsidian_url']

        return ifhtmx(
            request,
            Container(
                H1(out['title'], cls='uk-h1 text-4xl font-bold mt-12 mb-6'),
                Card(NotStr(apply_classes(html)))
            )
        )
    elif out['response_type'] == ResponseTypes.file:
        file = out['content']
        return FileResponse(file)
    elif out['response_type'] == ResponseTypes.blob:
        blob_data = out['content']
        if not isinstance(blob_data, (bytes, bytearray)):
            blob_data = bytes(blob_data)

        mime_type = guess_mime(f"file.{out['extension']}") or "application/octet-stream"
        extension = out['extension'].lower()
        
        # If not HTMX, return raw file
        if not is_htmx:
            return StreamingResponse(
                BytesIO(blob_data),
                media_type=mime_type,
                headers={
                    "Content-Disposition": f'inline; filename="{(title or "file")}.{extension.lstrip(".")}"',
                    "Cache-Control": "no-store",
                }
            )
        
        # For HTMX requests, embed as base64 data URLs
        import base64
        b64_data = base64.b64encode(blob_data).decode('utf-8')
        data_url = f"data:{mime_type};base64,{b64_data}"
        
        # Check file type and return appropriate HTML
        if extension in IMAGE_EXTS:
            return ifhtmx(
                request,
                Container(
                    H1(out['title'] or title or "Image", cls='uk-h1 text-4xl font-bold mt-12 mb-6'),
                    Card(
                        Img(src=data_url, alt=out['title'] or title or "Image", 
                            style="max-width: 100%; height: auto;",
                            cls="uk-responsive-width")
                    )
                )
            )
        elif extension in VIDEO_EXTS:
            return ifhtmx(
                request,
                Container(
                    H1(out['title'] or title or "Video", cls='uk-h1 text-4xl font-bold mt-12 mb-6'),
                    Card(
                        Video(
                            src=data_url,
                            controls=True,
                            style="max-width: 100%; height: auto;"
                        )
                    )
                )
            )
        elif extension in AUDIO_EXTS:
            return ifhtmx(
                request,
                Container(
                    H1(out['title'] or title or "Audio", cls='uk-h1 text-4xl font-bold mt-12 mb-6'),
                    Card(
                        Audio(
                            src=data_url,
                            controls=True
                        )
                    )
                )
            )
        else:
            # For other file types, return as download/stream
            return StreamingResponse(
                BytesIO(blob_data),
                media_type=mime_type,
                headers={
                    "Content-Disposition": f'inline; filename="{(title or "file")}.{extension.lstrip(".")}"',
                    "Cache-Control": "no-store",
                }
            )
    else:
        return ifhtmx(request, Div("No Response"))
# @rt
# def open(request: Request, session, file: str = "", title: Optional[str]=None):
#     VIDEO_EXTS = (".mp4", ".webm", ".ogg", ".mov", ".mkv")
#     AUDIO_EXTS = (".mp3", ".wav", ".ogg", ".m4a", ".flac")
#     IMAGE_EXTS = (".png", ".jpg", ".jpeg", ".gif", ".svg", ".webp")
#     TEXTLIKE_EXTS = (".md", ".qmd", ".canvas", ".base")
#     store = DBStore(config) if config.ENABLE_DB_MODE else FileStore(config)
#     out = store.query_file(file)
#     if out['content'] is None: return Div(f"Path not found: {urllib.parse.unquote(file)}")

#     # If it's a directory → list contents
#     if out['is_folder']:
#         items = []
#         for p in out['content']:
#             name = p['fname']
#             href = p['url']
#             items.append(Li(A(name, href=href)))
#         return ifhtmx(request,
#             Div(
#                 f"Listing for {out['title']}:",
#                 Ul(*items)
#             )
#         )
#     elif out['response_type'] == ResponseTypes.html:
#         html = out['content']
#         session['obsidian_url'] = out['obsidian_url']

#         return ifhtmx(
#             request,
#             Container(
#                 H1(out['title'], cls='uk-h1 text-4xl font-bold mt-12 mb-6'),
#                 Card(NotStr(apply_classes(html)))
#             )
#         )
#     elif out['response_type'] == ResponseTypes.file:
#         file = out['content']
#         return FileResponse(file)
#     elif out['response_type'] == ResponseTypes.blob:
#         blob_data = out['content']
#         if not isinstance(blob_data, (bytes, bytearray)):
#             blob_data = bytes(blob_data)


#         mime_type = guess_mime(f"file.{out['extension']}") or "application/octet-stream"
#         print(mime_type)
#         return StreamingResponse(
#             BytesIO(blob_data),
#             media_type=mime_type,
#             headers={
#                 "Content-Disposition": f'inline; filename="{(title or "file")}.{out["extension"].lstrip(".")}"',
#                 "Cache-Control": "no-store",
#         }
#     )
#     else:
#         return ifhtmx(request, Div("No Response"))

# @rt
# def open(request: Request, session, file: str = "", title: Optional[str]=None):
#     vault_path = config.OBSIDIAN_VAULT
#     file_path = resolve_note_path(vault_path, file)
#     if file_path is None: return Div(f"Path not found: {urllib.parse.unquote(file)}")

#     # If it's a directory → list contents
#     if file_path.is_dir():
#         items = []
#         for p in sorted(file_path.iterdir()):
#             rel_path = p.relative_to(vault_path)
#             href = MountPaths.open.to(file=rel_path)
#             items.append(Li(A(p.name, href=href)))
#         return ifhtmx(request,
#             Div(
#                 f"Listing for {file_path}:",
#                 Ul(*items)
#             )
#         )
#     # If it's a file → show content (assuming text)
#     if file_path.is_file():
#         if file_path.name.lower().endswith(".ipynb"):
#             return ifhtmx(
#                     request,
#                     Container(
#                         H1(file_path.stem, cls='uk-h1 text-4xl font-bold mt-12 mb-6'),
#                         render_nb(file_path)
#                 ))


#         op = ObsidianPage.from_file_path(file_path)
#         session['obsidian_url'] = op.obsidian_url

#         return ifhtmx(
#             request,
#             Container(
#                 H1(op.title, cls='uk-h1 text-4xl font-bold mt-12 mb-6'),
#                 Card(NotStr(apply_classes(op.html)))
#             )
#         )

In [ ]:
#| export
def iter_file(path, chunk_size=8192):
    with open(path, "rb") as f:
        while chunk := f.read(chunk_size):
            yield chunk

In [ ]:
fname = 'Test.m4a'
guess_mime(fname)
# mime = mimetypes.guess_type(fname); mime[0]
# FileResponse?

'audio/mp4'

In [ ]:
#| export
def CollapsibleBlocks(title, comp):
    return Div(
            Input(type='checkbox'),
            Div(title, cls='collapse-title font-semibold'),
            Div(comp, cls='collapse-content text-sm'),
            cls='collapse collapse-plus border'
        )

In [ ]:
to_xml(CollapsibleBlocks('T', 'c'))

'<div class="collapse collapse-plus border">\n  <input type="checkbox" class="uk-input ">\n  <div class="collapse-title font-semibold">T</div>\n  <div class="collapse-content text-sm">c</div>\n</div>\n'

In [ ]:
#| export

@rt
def embed(request: Request, file: str = "", ext: Optional[str] = None, title: Optional[str] = None):
    VIDEO_EXTS = {".mp4", ".webm", ".ogg", ".mov", ".mkv"}
    AUDIO_EXTS = {".mp3", ".wav", ".ogg", ".m4a", ".flac"}
    IMAGE_EXTS = {".png", ".jpg", ".jpeg", ".gif", ".svg", ".webp"}
    TEXTLIKE_EXTS = {".md", ".qmd", ".canvas", ".base"}

    store = DBStore(config) if config.ENABLE_DB_MODE else FileStore(config)
    out = store.query_file(file)
    if out["content"] is None: return Div(f"Path not found: {urllib.parse.unquote(file)}")
    if out["is_folder"]:
            items = []
            for p in out["content"]:
                name = p["fname"]
                href = p["url"]
                items.append(Li(A(name, href=href)))
            return ifhtmx(request, Div(f"Listing for {out['title']}:", Ul(*items)))
    if ext in IMAGE_EXTS:
            headers = {"Content-Type": f"image/{ext.lstrip('.')}"}
            if title: headers["X-Image-Title"] = title
            alt_text = pathlib.Path(file).stem
            headers["X-Image-Alt"] = alt_text
            if out["response_type"] == ResponseTypes.file: return FileResponse(out["content"], headers=headers)
            elif out["response_type"] == ResponseTypes.blob: return StreamingResponse(BytesIO(out["content"]), headers=headers)

    if ext in VIDEO_EXTS:
            mime = guess_mime(file) or f"video/{ext.lstrip('.')}"
            headers = {"Accept-Ranges": "bytes"}
            if out["response_type"] == ResponseTypes.file: return FileResponse(out["content"], media_type=mime, headers=headers, filename=title or file)
            elif out["response_type"] == ResponseTypes.blob: return StreamingResponse(BytesIO(out["content"]), media_type=mime, headers=headers)

        # --- Audio files ---
    if ext in AUDIO_EXTS:
        mime = guess_mime(file) or f"audio/{ext.lstrip('.')}"
        headers = {"Accept-Ranges": "bytes"}
        if out["response_type"] == ResponseTypes.file: return FileResponse(out["content"], media_type=mime, headers=headers, filename=title or file)
        elif out["response_type"] == ResponseTypes.blob: return StreamingResponse(BytesIO(out["content"]), media_type=mime, headers=headers)

    if out["response_type"] == ResponseTypes.html:
        return CollapsibleBlocks(
            Div(out['title']),
            NotStr(apply_classes(out["content"])),
        )

    return ifhtmx(request, Div("Unsupported file type"))
    # if file.lower().endswith(".ipynb"):
    #         return ifhtmx(
    #             request,
    #             Container(
    #                 H1(title or pathlib.Path(file).stem, cls="uk-h1 text-4xl font-bold mt-12 mb-6"),
    #                 render_nb(out["content"]),
    #             ),
    #         )
# @rt
# def embed(request: Request, file: str = "", ext:Optional[str]=None, title:Optional[str]=None):
#     VIDEO_EXTS = {".mp4", ".webm", ".ogg", ".mov", ".mkv"}
#     AUDIO_EXTS = {".mp3", ".wav", ".ogg", ".m4a", ".flac"}
#     IMAGE_EXTS = {".png", ".jpg", ".jpeg", ".gif", ".svg", ".webp"}
#     TEXTLIKE_EXTS = {".md", ".qmd", ".canvas", ".base"}
#     vault_path = config.OBSIDIAN_VAULT
#     file_path = resolve_note_path(vault_path, file)
#     print("Z1")
#     if file_path is None: return Div(f"Path not found: {urllib.parse.unquote(file)}")

#     # If it's a directory → list contents
#     if file_path.is_dir():
#         items = []
#         for p in sorted(file_path.iterdir()):
#             rel_path = p.relative_to(vault_path)
#             href = MountPaths.open.to(file=rel_path)
#             items.append(Li(A(p.name, href=href)))
#         return ifhtmx(request,
#             Div(
#                 f"Listing for {file_path}:",
#                 Ul(*items)
#             )
#         )
#     # If it's a file → show content (assuming text)
#     if file_path.is_file():
#         print("Z2")
#         if ext in IMAGE_EXTS:
#             headers = {'Content-Type': f"image/{ext[1:]}"}
            
#             # Add alt and title to headers if available/derivable
#             # 'title' is passed as a parameter to the embed function
#             if title:
#                 headers['X-Image-Title'] = title
            
#             # 'alt' is not passed directly to the embed function, so derive it from the file name
#             # This 'alt' would typically be used by the HTML <img> tag that references this image.
#             # Adding it as a custom header here provides it in the response, as per instruction.
#             alt_text = pathlib.Path(file).stem
#             headers['X-Image-Alt'] = alt_text
            
#             return Response(file_path.read_bytes(), headers=headers)

#         # --- Video files ---
#         if ext in VIDEO_EXTS:
#             # mime = f"video/{ext[1:]}" if ext else "video/mp4"
#             # mime = mimetypes.guess_type(file_path.name, strict=False)
#             mime = guess_mime(file_path.name)
#             print(mime)
#             headers = {"Accept-Ranges": "bytes"}  # Important for seeking
#             # return Response(file_path.read_bytes(), headers=headers)
#             return FileResponse(file_path, media_type=mime, headers=headers, filename=file_path.name)

#         # --- Audio files ---
#         if ext in AUDIO_EXTS:
#             # mime = f"audio/{ext[1:]}" if ext else "audio/mpeg"
#             # mime = mimetypes.guess_type(file_path.name, strict=False)
#             mime = guess_mime(file_path.name)
#             headers = {"Accept-Ranges": "bytes"}  # Important for seeking
#             # return Response(file_path.read_bytes(), headers=headers)
#             print(mime)
#             return FileResponse(file_path, media_type=mime, headers=headers, filename=file_path.name)

#         if file_path.name.lower().endswith(".ipynb"):
#             return ifhtmx(
#                     request,
#                     Container(
#                         H1(file_path.stem, cls='uk-h1 text-4xl font-bold mt-12 mb-6'),
#                         render_nb(file_path)
#                 ))
#         op = ObsidianPage.from_file_path(file_path)
#         print("Heeeo")
#         return CollapsibleBlocks(
#            Div(op.title), 
#            NotStr(apply_classes(op.html))
#         )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()